In [1]:
from sklearn.metrics.pairwise import euclidean_distances 
import tensorflow as tf
import numpy as np
import math
dtype=tf.float64
pi=tf.Variable(3.14,dtype=tf.float64)
def time_kernel(X1,X2,l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2):
#     x1=np.delete(X1,time_axis-1,1)
#     x2=np.delete(X2,time_axis-1,1)
    col=X1.shape[1]
    l1=tf.convert_to_tensor([l1_1,l1_2,l1_3,l1_4,l1_5],dtype=tf.float64)
    x1=X1[:,:col-1]
    x2=X2[:,:col-1]
    x1=tf.expand_dims(x1/l1,1)
    x2=tf.expand_dims(x2/l1,0)
    sub1=tf.square(x1-x2)
    r1=tf.reduce_sum(sub1,axis=-1)
    k1=sig1  * tf.exp(-0.5 * r1)
    
    t1=X1[:,col-1:]
    t2=X2[:,col-1:]
    t1=tf.expand_dims(t1/l2,1)
    t2=tf.expand_dims(t2/l2,0)
    sub2=tf.abs(t1-t2)
    r2=tf.reduce_sum(sub2,axis=-1)
    a=np.sqrt(3)
    k2=sig2*(1.0+a*r2)*tf.exp(-a*r2)
    
    return k1*k2

def time_kernel1(X1,X2,l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2):
#     x1=np.delete(X1,time_axis-1,1)
#     x2=np.delete(X2,time_axis-1,1)
    col=X1.shape[1]
    l1=tf.convert_to_tensor([l1_1,l1_2,l1_3,l1_4,l1_5],dtype=tf.float64)
    x1=X1[0::3,:col-1]
    x2=X2[0::3,:col-1]
    x1=tf.expand_dims(x1/l1,1)
    x2=tf.expand_dims(x2/l1,0)
    sub1=tf.square(x1-x2)
    r1=tf.reduce_sum(sub1,axis=-1)
    k1=sig1  * tf.exp(-0.5 * r1)
    
    t1=X1[:3,col-1:]
    t2=X2[:3,col-1:]
    t1=tf.expand_dims(t1/l2,1)
    t2=tf.expand_dims(t2/l2,0)
    sub2=tf.abs(t1-t2)
    r2=tf.reduce_sum(sub2,axis=-1)
    a=np.sqrt(3)
    k2=sig2*(1.0+a*r2)*tf.exp(-a*r2)
    
    return k1,k2

C:\Users\Yuzhang Ge\anaconda3\envs\newenv\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
class time_GPmodel():
    def __init__(self,data,kernel1,kernel2,l1,l2,sig1,sig2,likvar=0):
        self.X = data[0]
        self.Y = data[1]
        self.sigma2 = likvar
        self.kern = lambda X1,X2: kernel1(X1,X2,l1,l2,sig1,sig2)
        self.kern2=lambda X1,X2: kernel2(X1,X2,l1,l2,sig1,sig2)
    def coeff_creation(self):
        K1,K2=self.kern2(self.X,self.X)
        K1=K1+self.sigma2*tf.eye(290,290,dtype=tf.float64)
        L1=tf.linalg.cholesky(K1)
        trm=tf.linalg.triangular_solve(L1,tf.eye(290,290,dtype=tf.float64),lower=True)
        K1_inv=tf.linalg.triangular_solve(tf.transpose(L1),trm,lower=False)

        L2=tf.linalg.cholesky(K2)
        trm2=tf.linalg.triangular_solve(L2,tf.eye(3,3,dtype=tf.float64),lower=True)
        K2_inv=tf.linalg.triangular_solve(tf.transpose(L2),trm2,lower=False)

        op1=tf.linalg.LinearOperatorFullMatrix(K1_inv)
        op2=tf.linalg.LinearOperatorFullMatrix(K2_inv)

        self.K_inv=tf.linalg.LinearOperatorKronecker([op1,op2]).to_dense()
#         print(self.K_inv)
    def predict(self,Xtst,with_var=False):
        Kstar=self.kern(Xtst,self.X)
#         print(Kstar.shape)
        Kstarstar=self.kern(Xtst,Xtst)
        mu=tf.matmul(tf.matmul(Kstar,self.K_inv),self.Y)
        if with_var:
            var=tf.linalg.diag_part(Kstarstar)
            var-=tf.matmul(tf.matmul(Kstar,self.K_inv),tf.transpose(Kstar))
            var+=self.sigma2
            return mu,var
        else:
            return mu

In [3]:
import pickle 
file1 = open('X_time_2', 'rb')
file2 = open('Y_time_2', 'rb')
train_X=pickle.load(file1)
file1.close()
train_Y=pickle.load(file2)
file2.close()

In [56]:
import math as m
X=tf.convert_to_tensor(train_X[:870,:])
Y=tf.convert_to_tensor(train_Y[:870])
l1_1=tf.Variable(10.,dtype=tf.float64)
l1_2=tf.Variable(3.,dtype=tf.float64)
l1_3=tf.Variable(3.,dtype=tf.float64)
l1_4=tf.Variable(4.,dtype=tf.float64)
l1_5=tf.Variable(5.,dtype=tf.float64)
l1=tf.convert_to_tensor([l1_1,l1_2,l1_3,l1_4,l1_5],dtype=tf.float64)
l2=tf.Variable(2.,dtype=tf.float64)
sig1=tf.Variable(2.,dtype=tf.float64)
sig2=tf.Variable(2.,dtype=tf.float64)
var=tf.Variable(0.02,dtype=tf.float64)

In [5]:
Ky=time_kernel(X,X,l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2)
Ky=Ky+0.0*tf.eye(870,870,dtype=tf.float64)
K1,K2=time_kernel1(X,X,l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2)
K1=K1+0.0*tf.eye(290,290,dtype=tf.float64)
K2=K2+0.0*tf.eye(3,3,dtype=tf.float64)
L1=tf.linalg.cholesky(K1)
trm=tf.linalg.triangular_solve(L1,tf.eye(290,290,dtype=tf.float64),lower=True)
K1_inv=tf.linalg.triangular_solve(tf.transpose(L1),trm,lower=False)

L2=tf.linalg.cholesky(K2)
trm2=tf.linalg.triangular_solve(L2,tf.eye(3,3,dtype=tf.float64),lower=True)
K2_inv=tf.linalg.triangular_solve(tf.transpose(L2),trm2,lower=False)

op1=tf.linalg.LinearOperatorFullMatrix(K1_inv)
op2=tf.linalg.LinearOperatorFullMatrix(K2_inv)
Ky_inv=tf.linalg.LinearOperatorKronecker([op1,op2]).to_dense()
z=-0.5*tf.linalg.logdet(Ky)-0.5*tf.matmul(tf.transpose(Y),tf.matmul(Ky_inv,Y))-len(X)*tf.math.log(2*pi)

In [6]:
z

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[-1.39450878e+08]])>

In [ ]:
Ky=time_kernel(X,X,l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2)
#     Ky=Ky+0.01*tf.eye(870,870,dtype=tf.float64)
L = tf.linalg.cholesky(Ky)
S1 = tf.linalg.triangular_solve(L, Y, lower=True)
num_dims = tf.cast(tf.shape(Y)[0], L.dtype)
# p = -0.5 * tf.reduce_sum(tf.square(S1), 0)
# p -= 0.5*num_dims*tf.math.log(2*np.float64(np.pi))
# p -= tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L)))

S2 = tf.linalg.triangular_solve(tf.transpose(L), S1, lower=False)
    
z=-(tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L)))+0.5 * tf.matmul(tf.transpose(Y),S2)+num_dims*tf.math.log(2*pi))

In [7]:
z

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[-1.39450878e+08]])>

In [55]:
optimizer = tf.optimizers.Adam(learning_rate=0.01)

def my_function1(l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2,var):
    
    Ky=time_kernel(X,X,l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2)
    Ky=Ky+var*tf.eye(870,870,dtype=tf.float64)
    L = tf.linalg.cholesky(Ky)
    S1 = tf.linalg.triangular_solve(L, Y, lower=True)
    num_dims = tf.cast(tf.shape(Y)[0], L.dtype)
    p = -0.5 * tf.reduce_sum(tf.square(S1), 0)
    p -= 0.5*num_dims*tf.math.log(2*np.float64(np.pi))
    p -= tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L)))
#    S2 = tf.linalg.triangular_solve(tf.transpose(L), S1, lower=False)
    
#    z=tf.reduce_sum(tf.math.log(tf.linalg.tensor_diag_part(L)))+0.5 * tf.matmul(tf.transpose(Y),S2)+num_dims*tf.math.log(2*np.float64(np.pi))
    return -p

In [57]:
def my_function2(l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2,var):
    
    Ky=time_kernel(X,X,l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2)
    Ky=Ky+0.0*tf.eye(870,870,dtype=tf.float64)
    K1,K2=time_kernel1(X,X,l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2)
    K1=K1+0.0*tf.eye(290,290,dtype=tf.float64)
    K2=K2+0.0*tf.eye(3,3,dtype=tf.float64)
    L1=tf.linalg.cholesky(K1)
    trm=tf.linalg.triangular_solve(L1,tf.eye(290,290,dtype=tf.float64),lower=True)
    K1_inv=tf.linalg.triangular_solve(tf.transpose(L1),trm,lower=False)

    L2=tf.linalg.cholesky(K2)
    trm2=tf.linalg.triangular_solve(L2,tf.eye(3,3,dtype=tf.float64),lower=True)
    K2_inv=tf.linalg.triangular_solve(tf.transpose(L2),trm2,lower=False)

    op1=tf.linalg.LinearOperatorFullMatrix(K1_inv)
    op2=tf.linalg.LinearOperatorFullMatrix(K2_inv)
    Ky_inv=tf.linalg.LinearOperatorKronecker([op1,op2]).to_dense()
    Ky_inv=Ky_inv+(var**2)*tf.eye(870,870,dtype=tf.float64)

    z=-0.5*tf.linalg.logdet(Ky)-0.5*tf.matmul(tf.transpose(Y),tf.matmul(Ky_inv,Y))-len(X)*tf.math.log(2*pi)
    return -z

In [58]:
for i in range(10000):
    optimizer.minimize(lambda:my_function2(l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2,var), var_list=[l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2,var])

In [59]:
[l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2,var]

[<tf.Variable 'Variable:0' shape=() dtype=float64, numpy=8.40976967101872>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=0.08252057197259115>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=1.0856217685726792>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=2.4010857004321546>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=3.148471610968141>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=0.25200866335388844>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=3.3355282212023454>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=3.3355282212340276>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=-2.647439957099227e-11>]

In [43]:
learning_rate = 0.1
optimizer = tf.keras.optimizers.SGD(learning_rate)
# Define the training loop
for i in range(1):
    # Use tf.GradientTape to record the gradients of the loss with respect to the variables
    with tf.GradientTape() as tape:
        loss = my_function2(l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2,var)
    gradients = tape.gradient(loss, [l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2,var])
    
    # Use the optimizer to update the variables
    optimizer.apply_gradients(zip(gradients, [l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2,var]))

In [44]:
[l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2,var]

[<tf.Variable 'Variable:0' shape=() dtype=float64, numpy=nan>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=nan>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=nan>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=nan>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=nan>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=nan>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=nan>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=nan>,
 <tf.Variable 'Variable:0' shape=() dtype=float64, numpy=-65954294.43826243>]

In [62]:
my_function2(l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2,var)

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[3221.66110175]])>

In [61]:
my_function2(6.33902592,  0.58283949, 13.33258808,  2.06577564,  2.13873601,0.39523340421,6.65476945854,6.65476945854,0.00034323097719074833)

<tf.Tensor: shape=(1, 1), dtype=float64, numpy=array([[1850.15982197]])>

In [15]:

# Define the training loop
for i in range(100):
    # Use tf.GradientTape to record the gradients of the loss with respect to the variables
    with tf.GradientTape() as tape:
        loss = my_function2(l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2)
    gradients = tape.gradient(loss, [l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2])
    
    # Use the optimizer to update the variables
    optimizer.apply_gradients(zip(gradients, [l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2]))

In [44]:
my_function1(l1_1,l1_2,l1_3,l1_4,l1_5,l2,sig1,sig2,var)

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([1519.07889491])>

In [45]:
my_function1(6.33902592,  0.58283949, 13.33258808,  2.06577564,  2.13873601, 0.395233404,6.654769477,6.654769458,0.00034323097719074833)

<tf.Tensor: shape=(1,), dtype=float64, numpy=array([1050.8935768])>

In [ ]:
import tensorflow as tf

# Define your self-designed function with multiple variables
def my_func(x, y, z):
    return tf.sin(x) + tf.square(y) + tf.exp(z)

# Initialize your variables
x = tf.Variable(1.0)
y = tf.Variable(2.0)
z = tf.Variable(3.0)

# Define the optimizer and the learning rate
optimizer = tf.optimizers.Adam(learning_rate=0.1)

# Define the training loop
for i in range(1000):
    # Use tf.GradientTape to record the gradients of the loss with respect to the variables
    with tf.GradientTape() as tape:
        loss = my_func(x, y, z)
    gradients = tape.gradient(loss, [x, y, z])
    
    # Use the optimizer to update the variables
    optimizer.apply_gradients(zip(gradients, [x, y, z]))
print("Final values: x = {}, y = {}".format(x.numpy(), y.numpy()))  

In [ ]:
my_func(x, y, z)

In [ ]:
import tensorflow as tf

# Define the function to optimize
def my_function(x, y):
    return x**2 + y**2

# Create a TensorFlow variable for each input
x = tf.Variable(2.0)
y = tf.Variable(3.0)

# Define the Adam optimizer with a learning rate of 0.01
adam = tf.keras.optimizers.Adam(learning_rate=0.01)

# Compile the function with the Adam optimizer
@tf.function
def my_function_optimized():
    loss = my_function(x, y)
    grads = tf.gradients(loss, [x, y])
    adam.apply_gradients(zip(grads, [x, y]))
    return loss

# Optimize the function for 100 iterations
for i in range(100):
    loss = my_function_optimized()
    print("Iteration {}: loss = {}".format(i+1, loss))

# Print the final values of x and y
print("Final values: x = {}, y = {}".format(x.numpy(), y.numpy()))